In [2]:
pip install selenium

     ---------------------------------------- 7.2/7.2 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 400.2/400.2 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing necessary libraries
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
class Weather_API:
    
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=4e70cabb80884db08524a28ac33cdc1d'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

In [6]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        # Start the Driver
        driver = webdriver.Chrome(executable_path = r"C:\Users\finaz\Taiyo.ai\chromedriver.exe")
        
        # Hit the url of NASA Earth Data website and wait for 15 seconds.
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
        display(df)
        
        # Store to csv file
        df.to_csv('ws.csv', sep=',', index=False,header=True)
        
        print('Web Scraping Successful!')

In [7]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Keyword to be searched: mumbai
API call successful!
{
    "articles": [
        {
            "author": "https://www.facebook.com/bbcnews",
            "content": "A constable with India's Railway Protection Force (RPF) has been arrested after he shot dead four people on a train near the city of Mumbai.\r\nPolice said the accused, Chetan Kumar, killed a senior RP\u2026 [+1172 chars]",
            "description": "The man was arrested by the railway police and his motive is being investigated.",
            "publishedAt": "2023-07-31T07:18:24Z",
            "source": {
                "id": "bbc-news",
                "name": "BBC News"
            },
            "title": "Jaipur-Mumbai train: India railway constable shoots four dead on train",
            "url": "https://www.bbc.co.uk/news/world-asia-india-66337287",
            "urlToImage": "https://ichef.bbci.co.uk/news/1024/branded_news/16B5B/production/_130591039_anitrain.jpg"
        },
        {
            "author": "The A

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,bbc-news,BBC News,https://www.facebook.com/bbcnews,Jaipur-Mumbai train: India railway constable s...,The man was arrested by the railway police and...,https://www.bbc.co.uk/news/world-asia-india-66...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-07-31T07:18:24Z,A constable with India's Railway Protection Fo...
1,None,NPR,The Associated Press,Indian rescuers find 27 dead and dozens are mi...,Seventy-eight people are still unaccounted for...,https://www.npr.org/2023/07/23/1189660003/indi...,https://media.npr.org/assets/img/2023/07/23/ap...,2023-07-23T11:41:02Z,Rescuers walk towards the site of a landslide ...
2,None,The Guardian,Tim Adams,The big picture: Danish Siddiqui’s vivid image...,The late war photographer’s atmospheric portra...,https://www.theguardian.com/artanddesign/2023/...,https://i.guim.co.uk/img/media/c2fdd237e85d77f...,2023-08-06T06:00:04Z,The photographer Danish Siddiqui joined the Re...
3,None,The Guardian,Alia Waheed,Student’s sexual assault and murder shows wome...,"Despite reforms, campaigners say Mumbai killin...",https://www.theguardian.com/global-development...,https://i.guim.co.uk/img/media/e4bf225ee92321b...,2023-07-27T05:00:27Z,The list of complaints made by female students...
4,None,The Guardian,PA Media,Jofra Archer on course to be fit for England’s...,<ul><li>Fast bowler making progress after long...,https://www.theguardian.com/sport/2023/jul/26/...,https://i.guim.co.uk/img/media/3362bee5db68916...,2023-07-26T20:53:37Z,Jofra Archer is on course to be fit for Englan...


Enter Location: india


TypeError: __init__() got an unexpected keyword argument 'executable_path'

In [10]:
data = pd.read_csv('ws.csv')
data

,title,description,link
0,"Brahmaputra River, India | Earthdata",...visualizations of the Brahmaputra River in ...,https://earthdata.nasa.gov/worldview/worldview...
1,Fires in Northwest India | Earthdata,...visualizations of fires in northwest India ...,https://earthdata.nasa.gov/worldview/worldview...
2,High Aerosol Index Over Northern India | Earth...,"...of high aerosol index over Northern India, ...",https://earthdata.nasa.gov/worldview/worldview...
3,Fires and Smoke in Northern India | Earthdata,...visualizations of fires and smoke in North ...,https://earthdata.nasa.gov/worldview/worldview...
4,Tropical Cyclone Vardah approaching India | Ea...,...visualizations of Tropical Cyclone Vardah n...,https://earthdata.nasa.gov/worldview/worldview...
...,...,...,...
231,[PDF],...1035 24.96% CHINA 348 8.39% 394 9.50% INDIA...,https://cdn.earthdata.nasa.gov/conduit/upload/...
232,[PDF],...that pours on the Bay of Bengal in India. “...,https://cdn.earthdata.nasa.gov/conduit/upload/...
233,[PDF],"...instance, the ocean surface off the tip of ...",https://cdn.earthdata.nasa.gov/conduit/upload/...
234,[PDF],"...in the 40 spotlight in recent years, India ...",https://cdn.earthdata.nasa.gov/conduit/upload/...
